In [1]:
from pymongo import MongoClient
import configparser
import pandas as pd
import re
from nltk.corpus import wordnet
import numpy as np
from flask import Flask, render_template
# import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

### 資料處理

In [2]:
_df = pd.read_csv("./data.csv")

In [3]:
# 建立字典，將年月份對應到角度
month_labels = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
               '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01']
angle_mapping = [(i * 15) % 360 for i in range(len(month_labels))]
dict_date_angle = dict(zip(month_labels, angle_mapping))


In [4]:
def transform(text):
    text = str(text)
    year = text[:4]
    month = text[-2:]
    return year + "-" + month

In [5]:
# 將日期轉換為角度
def date_to_angle(date):
    date = date.strftime("%Y-%m-%d")
    return dict_date_angle[date]

In [6]:
_df['monitormonth'] = pd.to_datetime(_df["monitormonth"].apply(lambda x: transform(x)), format='%Y-%m')

In [7]:
_df

siteid sitename  itemid  itemname itemengname itemunit monitormonth  \
0            6       板橋       1      二氧化硫         SO2      ppb   1982-07-01   
1            6       板橋      38      相對濕度          RH  percent   1982-07-01   
2           67       三重       5      氮氧化物         NOx      ppb   1982-07-01   
3            6       板橋       9  非甲烷碳氫化合物        NMHC      ppm   1982-07-01   
4            6       板橋       7      二氧化氮         NO2      ppb   1982-07-01   
...        ...      ...     ...       ...         ...      ...          ...   
366765      85       大城       6      一氧化氮          NO      ppb   2023-04-01   
366766      85       大城       7      二氧化氮         NO2      ppb   2023-04-01   
366767      85       大城       8     總碳氫化物         THC      ppm   2023-04-01   
366768      85       大城       9  非甲烷碳氫化合物        NMHC      ppm   2023-04-01   
366769      85       大城      10        風速  WIND_SPEED    m/sec   2023-04-01   

       concentration  
0               40.6  
1              15.86  
2             189.54  
3                  x  
4                 23  
...              ...  
366765           1.1  
366766           6.8  
366767          2.22  
366768          0.01  
366769           3.5  

[366770 rows x 8 columns]

In [17]:
# find the min of monitormonth
min_date = _df['monitormonth'].min()
max_date = _df['monitormonth'].max()
print(min_date.strftime('%Y-%m-%d'), "~", max_date.strftime('%Y-%m-%d'))

1982-07-01 ~ 2023-04-01


In [8]:
start_date = pd.to_datetime('2021-01', format='%Y-%m')
end_date = pd.to_datetime('2022-12', format='%Y-%m')
filtered_df = _df[(_df['monitormonth'] >= start_date) & (_df['monitormonth'] <= end_date)]

In [21]:
filtered_df['sitename'].unique().tolist()

['富貴角',
 '麥寮',
 '關山',
 '馬公',
 '金門',
 '馬祖',
 '埔里',
 '復興',
 '永和',
 '竹山',
 '中壢',
 '三重',
 '冬山',
 '宜蘭',
 '陽明',
 '花蓮',
 '臺東',
 '恆春',
 '潮州',
 '屏東',
 '小港',
 '前鎮',
 '前金',
 '左營',
 '楠梓',
 '林園',
 '大寮',
 '鳳山',
 '仁武',
 '橋頭',
 '美濃',
 '臺南',
 '安南',
 '善化',
 '新營',
 '嘉義',
 '臺西',
 '朴子',
 '新港',
 '崙背',
 '斗六',
 '南投',
 '二林',
 '線西',
 '彰化',
 '西屯',
 '忠明',
 '大里',
 '沙鹿',
 '豐原',
 '三義',
 '苗栗',
 '頭份',
 '新竹',
 '竹東',
 '湖口',
 '龍潭',
 '平鎮',
 '觀音',
 '大園',
 '桃園',
 '大同',
 '松山',
 '古亭',
 '萬華',
 '中山',
 '士林',
 '淡水',
 '林口',
 '菜寮',
 '新莊',
 '板橋',
 '土城',
 '新店',
 '萬里',
 '汐止',
 '基隆',
 '大城']

In [12]:
loc = filtered_df[filtered_df['sitename']=="埔里"]
loc = loc.replace('x', 0)
loc = loc.fillna(0)

In [13]:
loc

siteid sitename  itemid itemname itemengname itemunit monitormonth  \
336080      72       埔里     143    小時風速值       WS_HR    m/sec   2021-01-01   
336081      72       埔里      38     相對濕度          RH        %   2021-01-01   
336082      72       埔里      33    細懸浮微粒       PM2.5    μg/m3   2021-01-01   
336083      72       埔里      14       溫度    AMB_TEMP        ℃   2021-01-01   
336084      72       埔里      10       風速  WIND_SPEED    m/sec   2021-01-01   
...        ...      ...     ...      ...         ...      ...          ...   
362277      72       埔里      14       溫度    AMB_TEMP        ℃   2022-12-01   
362278      72       埔里      31       甲烷         CH4      ppm   2022-12-01   
362279      72       埔里      33    細懸浮微粒       PM2.5    μg/m3   2022-12-01   
362280      72       埔里      38     相對濕度          RH        %   2022-12-01   
362281      72       埔里     143    小時風速值       WS_HR    m/sec   2022-12-01   

       concentration  
336080             1  
336081            72  
336082            22  
336083          15.2  
336084           1.4  
...              ...  
362277            18  
362278           2.1  
362279            12  
362280            69  
362281           0.7  

[330 rows x 8 columns]

In [14]:
pd.pivot_table(loc, values='concentration', index='monitormonth', columns='itemengname')

itemengname   AMB_TEMP   CH4    CO  NMHC   NO   NO2   NOx    O3  PM10  PM2.5  \
monitormonth                                                                   
2021-01-01        15.2   NaN  0.41   NaN  2.6  11.7  14.3  26.3  31.0   22.0   
2021-02-01        18.5   NaN  0.39   NaN  1.9  11.4  13.3  36.8  39.0   26.0   
2021-03-01        21.6   NaN  0.38   NaN  1.8  10.9  12.7  35.2  39.0   25.0   
2021-04-01        23.6   NaN  0.34   NaN  1.7   9.9  11.6  79.9  38.0   24.0   
2021-05-01        27.4   NaN  0.21   NaN  2.0   9.1  11.1  31.6  24.0   13.0   
2021-06-01        25.8   NaN  0.16   NaN  1.6   5.3   7.0  22.4  16.0    5.0   
2021-07-01        27.4   NaN  0.14   NaN  1.1   3.9   5.2  26.0  15.0    8.0   
2021-08-01        26.2   NaN  0.16   NaN  1.4   4.4   5.9  20.8  14.0    8.0   
2021-09-01        27.2   NaN  0.21   NaN  1.4   4.9   6.4  31.9  22.0   14.0   
2021-10-01        25.6   NaN  0.27   NaN  1.5   6.7   8.3  33.8  27.0   17.0   
2021-11-01        21.6  0.00  0.32  0.00  1.9   8.5  10.5  25.6  25.0   16.0   
2021-12-01        18.1  0.00  0.38  0.00  2.3  10.0  12.4  23.3  28.0   18.0   
2022-01-01        17.8  2.09  0.36  0.11  2.1   9.1  11.3  21.8  24.0   17.0   
2022-02-01        17.4  2.08  0.32  0.09  1.9   7.5   9.6  22.1  19.0   15.0   
2022-03-01        21.6  2.10  0.31  0.09  1.8   7.0   8.9  31.3  29.0   17.0   
2022-04-01        23.3  2.06  0.26  0.06  1.4   5.4   6.9  32.4  27.0   16.0   
2022-05-01        24.4  2.06  0.22  0.06  1.5   4.4   6.0  23.9  18.0    7.0   
2022-06-01        25.9  2.00  0.15  0.04  1.6   2.8   4.5  16.0  14.0    3.0   
2022-07-01        27.9  2.00  0.16  0.03  1.3   2.8   4.2  23.4  16.0    5.0   
2022-08-01        27.5  2.04  0.17  0.04  1.3   3.1   4.5  25.0  16.0    5.0   
2022-09-01        26.6  2.12  0.25  0.05  1.3   3.9   5.3  35.1  26.0   12.0   
2022-10-01        25.2  2.11  0.26  0.07  1.6   5.6   7.3  30.6  28.0   13.0   
2022-11-01        24.3  2.12  0.30  0.09  1.8   6.5   8.4  26.4  27.0   13.0   
2022-12-01        18.0  2.10  0.33  0.11  2.3   8.5  10.8  22.8  22.0   12.0   

itemengname     RH  SO2   THC  WIND_SPEED  WS_HR  
monitormonth                                      
2021-01-01    72.0  1.4   NaN         1.4    1.0  
2021-02-01    67.0  1.5   NaN         1.6    1.1  
2021-03-01    71.0  1.5   NaN         1.7    1.3  
2021-04-01    70.0  1.5   NaN         1.7    1.3  
2021-05-01    69.0  1.4   NaN         1.9    1.4  
2021-06-01    80.0  1.3   NaN         1.4    0.9  
2021-07-01    76.0  1.1   NaN         1.6    1.2  
2021-08-01    82.0  1.1   NaN         1.2    0.9  
2021-09-01    75.0  1.3   NaN         1.6    1.2  
2021-10-01    72.0  1.3   NaN         1.4    1.1  
2021-11-01    77.0  1.4  0.00         1.1    0.8  
2021-12-01    77.0  1.3  0.00         1.1    0.8  
2022-01-01    79.0  1.2  2.21         1.1    0.8  
2022-02-01    83.0  1.1  2.18         0.9    0.7  
2022-03-01    74.0  1.3  2.20         1.1    0.9  
2022-04-01    74.0  0.7  2.14         1.4    1.1  
2022-05-01    85.0  1.1  2.13         1.0    0.7  
2022-06-01    83.0  1.1  2.04         1.1    0.7  
2022-07-01    74.0  1.1  2.04         1.4    1.0  
2022-08-01    73.0  1.2  2.09         1.4    1.0  
2022-09-01    70.0  1.1  2.18         1.3    1.0  
2022-10-01    71.0  1.2  2.19         1.1    0.8  
2022-11-01    71.0  1.2  2.22         0.9    0.6  
2022-12-01    69.0  1.3  2.21         0.9    0.7

In [15]:
loc = filtered_df[filtered_df['sitename']=="臺南"]
loc = loc.replace('x', 0)
loc = loc.fillna(0)
loc = pd.pivot_table(loc, values='concentration', index=loc['monitormonth'], columns='itemengname')
loc = loc.reset_index()
df_location = loc[['monitormonth', 'PM2.5', 'PM10', 'CO', 'RH']]
df_location.rename(columns = {'RH':'humidity'}, inplace = True)
df_location['angle'] = df_location.monitormonth.apply(lambda x: date_to_angle(x))

c:\Users\User\anaconda3\envs\text\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\Users\User\anaconda3\envs\text\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
df_location

itemengname monitormonth  PM2.5  PM10    CO  humidity  angle
0             2021-01-01   37.0  60.0  0.49      72.0      0
1             2021-02-01   41.0  69.0  0.47      71.0     15
2             2021-03-01   37.0  69.0  0.45      71.0     30
3             2021-04-01   24.0  56.0  0.37      70.0     45
4             2021-05-01   11.0  25.0  0.22      71.0     60
5             2021-06-01    6.0  15.0  0.17      78.0     75
6             2021-07-01    7.0  17.0  0.16      78.0     90
7             2021-08-01    7.0  15.0  0.20      87.0    105
8             2021-09-01   13.0  25.0  0.23      80.0    120
9             2021-10-01   16.0  38.0  0.31      72.0    135
10            2021-11-01   21.0  50.0  0.36      72.0    150
11            2021-12-01   27.0  54.0  0.39      73.0    165
12            2022-01-01   27.0  45.0  0.40      78.0    180
13            2022-02-01   24.0  38.0  0.36      80.0    195
14            2022-03-01   30.0  51.0  0.39      74.0    210
15            2022-04-01   21.0  38.0  0.27      74.0    225
16            2022-05-01   14.0  30.0  0.24      81.0    240
17            2022-06-01    6.0  15.0  0.13      76.0    255
18            2022-07-01    6.0  15.0  0.13      74.0    270
19            2022-08-01    8.0  15.0  0.18      77.0    285
20            2022-09-01    0.0   0.0  0.00      70.0    300
21            2022-10-01   18.0  39.0  0.42      70.0    315
22            2022-11-01   26.0  45.0  0.54      82.0    330
23            2022-12-01   21.0  38.0  0.48      70.0    345

In [39]:
import plotly.graph_objects as go

# 自定義的年月份標籤
month_labels = ['2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06',
               '2021/07', '2021/08', '2021/09', '2021/10', '2021/11', '2021/12',
               '2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
               '2022/07', '2022/08', '2022/09', '2022/10', '2022/11', '2022/12']

# 創建 Polar Charts
fig = go.Figure()

# 添加數據到圖表，並設定標籤文本
# fig.add_trace(go.Scatterpolar(
#     name='相對濕度',
#     r=df_location['相對濕度'].tolist(),
#     theta=month_labels,
#     mode='markers',  # 使用 'lines+markers' 顯示線段和點
#     marker=dict(
#         size=10,
#         color=df_location['相對濕度'].tolist(),  # 根據 r 值設定顏色
#         colorscale='Blues',  # 設定顏色漸層
#         # colorbar=dict(
#         #     title='相對濕度'  # 設定顏色軸標題
#         # ),
#         cmin=0.9*min(df_location['相對濕度'].tolist()),  # 設定顏色漸層最小值
#         cmax=max(df_location['相對濕度'].tolist())  # 設定顏色漸層最大值
#     ),
    
#     line=dict(
#         color='blue',
#         width=1
#     ),
#     text=month_labels,  # 設定標籤文本為年月份
#     hovertemplate="日期: %{text}<br>percent: %{r} %<br>",  # 定義 hover 顯示的文本格式
# ))
fig.add_trace(go.Scatterpolar(
    name='細懸浮微粒',
    r=df_location['相對濕度'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        color=df_location['細懸浮微粒'].tolist(),  # 根據 r 值設定顏色
        # symbol=df_location['細懸浮微粒'].tolist(),
        colorscale='Reds',  # 設定顏色漸層
        size = 10,
        colorbar=dict(
            title='細懸浮微粒'  # 設定顏色軸標題
        ),
        cmin=0.9*min(df_location['細懸浮微粒'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['細懸浮微粒'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    # text=month_labels,  # 設定標籤文本為年月份
    # hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式
))

# 設定角度刻度和標籤，並且顏色填充用藍色根據r值由小到大漸層
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=0,
            tickvals=angle_mapping,
            ticks="outside",
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    ),
    height=600,
    # width=600,
    # showlegend=False
)

# 顯示圖表
fig.show()

In [19]:
import plotly.graph_objects as go

# 自定義的年月份標籤
month_labels = ['2021/01', '2021/02', '2021/03', '2021/04', '2021/05', '2021/06',
               '2021/07', '2021/08', '2021/09', '2021/10', '2021/11', '2021/12',
               '2022/01', '2022/02', '2022/03', '2022/04', '2022/05', '2022/06',
               '2022/07', '2022/08', '2022/09', '2022/10', '2022/11', '2022/12']

# 創建 Polar Charts
fig = go.Figure()

# 添加數據到圖表，並設定標籤文本
fig.add_trace(go.Scatterpolar(
    name='humidity',
    r=df_location['humidity'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['humidity'].tolist(),  # 根據 r 值設定顏色
        colorscale='Blues',  # 設定顏色漸層
        colorbar=dict(
            title='humidity',
            len=0.4,
            y=0.25,
            x=1.1,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=0.9*min(df_location['humidity'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['humidity'].tolist())  # 設定顏色漸層最大值
    ),
    
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>percent: %{r} %<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='PM2.5',
    r=df_location['PM2.5'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['PM2.5'].tolist(),  # 根據 r 值設定顏色
        colorscale='Reds',  # 設定顏色漸層,
        colorbar=dict(
            title='PM2.5',
            len=0.4,
            y=0.25,
            x=0.9,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=0.9*min(df_location['PM2.5'].tolist()),  # 設定顏色漸層最小值
        cmax=max(df_location['PM2.5'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式
))
fig.add_trace(go.Scatterpolar(
    name='PM10',
    r=df_location['PM10'].tolist(),
    theta=month_labels,
    mode='markers',  # 使用 'lines+markers' 顯示線段和點
    marker=dict(
        size=10,
        color=df_location['PM10'].tolist(),  # 根據 r 值設定顏色
        colorscale='Greens',  # 設定顏色漸層
        colorbar=dict(
            title='PM10',
            len=0.4,
            y=0.25,
            x=1,
            tickfont=dict(
                size=9,
            ),
            xpad=2
        ),
        cmin=min(df_location['PM10'].tolist()),  # 設定顏色漸層最小值
        cmax=0.9*max(df_location['PM10'].tolist())  # 設定顏色漸層最大值
    ),
    line=dict(
        color='blue',
        width=1
    ),
    text=month_labels,  # 設定標籤文本為年月份
    hovertemplate="日期: %{text}<br>μg/m3: %{r}<br>",  # 定義 hover 顯示的文本格式 
))

# 設定角度刻度和標籤，並且顏色填充用藍色根據r值由小到大漸層
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            angle=0,
            tickvals=angle_mapping,
            ticks="outside",
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    ),
    height=600,
    # coloraxis_colorbar=colorbar
)

# 顯示圖表
fig.show()

In [ ]:
import plotly.graph_objects as go

# 設定角度刻度和標籤
angles = [i * 15 for i in range(24)]  # 每十五度一個刻度
labels = [str(i) + "°" for i in angles]  # 刻度標籤

# 創建 Polar Chart
fig = go.Figure()

# 添加從中心往外的線
for angle in angles:
    fig.add_trace(go.Scatterpolar(
        r=[0, 1],  # 從中心 (r=0) 繪製到半徑為 1 的線
        theta=[angle, angle],  # 角度起點和終點相同，畫一條直線
        mode='lines',
        line=dict(
            color='black',
            width=1
        ),
        hoverinfo='skip'  # 不顯示 hover 資訊
    ))

# 設定角度刻度和標籤
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=False  # 隱藏半徑軸
        ),
        angularaxis=dict(
            tickvals=angles,
            ticktext=labels,
            tickmode='array',
            tickfont=dict(
                size=8,
            )
        )
    )
)

# 顯示圖表
fig.show()
